In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join network example 8 - fork-join with multiple visits within the same chain
model = Network('model')

In [3]:
# Create network
source = Source(model,'Source')
queue1 = Queue(model,'Queue1',SchedStrategy.PS)
queue2 = Queue(model,'Queue2',SchedStrategy.PS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)
sink = Sink(model,'Sink')

jobclass1 = OpenClass(model, 'class1')
jobclass2 = OpenClass(model, 'class2')

In [4]:
# Service configurations
source.set_arrival(jobclass1, Exp(0.1))
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(1.0))
queue1.set_service(jobclass2, Exp(1.0))
queue2.set_service(jobclass2, Exp(1.0))

In [5]:
# Set routing matrix with class switching - multiple visits
P = model.init_routing_matrix()

# Class 1 initial routing
P.set(jobclass1, jobclass1, source, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)

# Class switching: class1 -> class2 after join
P.set(jobclass1, jobclass2, join, fork, 1.0)

# Class 2 routing
P.set(jobclass2, jobclass2, fork, queue1, 1.0)
P.set(jobclass2, jobclass2, fork, queue2, 1.0)
P.set(jobclass2, jobclass2, queue1, join, 1.0)
P.set(jobclass2, jobclass2, queue2, join, 1.0)
P.set(jobclass2, jobclass2, join, sink, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [SolverMVA(model), SolverJMT(model, seed=23000)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.get_avg_table()

SOLVER 1: SolverMVA
MVA analysis [method: default(egflin), lang: java, env: 21.0.8] completed in 0.056494s.
  Station JobClass    QLen  Util   RespT  ResidT  ArvR  Tput
0  Source   class1  0.0000   0.0  0.0000  0.0000  0.00   0.1
2  Queue1   class1  0.1247   0.1  1.2473  0.6236  0.10   0.1
3  Queue1   class2  0.1247   0.1  1.2473  0.6236  0.05   0.1
4  Queue2   class1  0.1247   0.1  1.2473  0.6236  0.10   0.1
5  Queue2   class2  0.1247   0.1  1.2473  0.6236  0.05   0.1
6    Join   class1  0.1247   0.0  0.6236  0.6236  0.20   0.1
7    Join   class2  0.1247   0.0  0.6236  0.6236  0.10   0.1
SOLVER 2: SolverJMT
JMT Model: /tmp/workspace/jsim/1961779642320289935/jmodel.jsim
JMT [method: default, lang: java, env: 21.0.8] completed in 1.756938s.
  Station JobClass    QLen    Util   RespT  ResidT    ArvR    Tput
0  Source   class1  0.0000  0.0000  0.0000  0.0000  0.0000  0.1021
2  Queue1   class1  0.1306  0.1015  1.2685  0.6343  0.1021  0.1021
3  Queue1   class2  0.1353  0.0992  1.3002  0.650